In [1]:
import os
import pandas as pd
import numpy as np
import os
import psycopg2 # PostgreSQL database adapter for Python
from dotenv import load_dotenv # Reads the key-value pair from .env file and adds them to environment variable
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load environment variables from .env file
load_dotenv()

# Accessing credentials
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")

In [3]:
# Connect to the database
conn = psycopg2.connect(
    host=db_host,
    dbname=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)

In [4]:
query_main=query="""
    select * 
    from agg.tidy_data_30min
    where site = 20
"""
df_main = pd.read_sql_query(query_main, conn)

In [5]:
pd.set_option('display.max_columns', None)
df_main = df_main[['timestamp','net_load','month', 'day', 'hour', 'day_of_week', 'season', 'avg_net_load','weekend_or_bank_holiday','solar_radiation','sunshine_duration']]
df_main
df_main

,timestamp,net_load,month,day,hour,day_of_week,season,avg_net_load,weekend_or_bank_holiday,solar_radiation,sunshine_duration
0,2019-04-23 15:00:00+00:00,9.38,4,23,16,1,2,-27.63,0,NaN,NaN
1,2019-04-23 15:30:00+00:00,10.35,4,23,16,1,2,-24.74,0,NaN,NaN
2,2019-04-23 16:00:00+00:00,16.79,4,23,17,1,2,5.20,0,NaN,NaN
3,2019-04-23 16:30:00+00:00,16.98,4,23,17,1,2,28.17,0,NaN,NaN
4,2019-04-23 17:00:00+00:00,11.89,4,23,18,1,2,53.22,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
17557,2020-03-30 04:30:00+00:00,511.06,3,30,5,0,2,608.02,0,NaN,NaN
17558,2020-03-30 05:00:00+00:00,320.69,3,30,6,0,2,586.85,0,NaN,NaN
17559,2020-03-30 05:30:00+00:00,245.21,3,30,6,0,2,524.16,0,NaN,NaN
17560,2020-03-30 06:00:00+00:00,187.68,3,30,7,0,2,458.60,0,NaN,NaN


In [6]:
#cheking the range of df_ma'timestamp']
print(df_main['timestamp'].max())

2020-03-31 22:30:00+00:00


In [7]:
#Set view format to scrollable for better visualization
pd.set_option('display.max_rows', 200)

In [15]:
# Check the range of the timestamp column
min_timestamp = df_main['timestamp'].min()
max_timestamp = df_main['timestamp'].max()

# Get the last 25 values of the timestamp range
last_25_timestamps = df_main['timestamp'].sort_values().tail(200)
last_25_timestamps

17370   2020-03-27 21:00:00+00:00
17371   2020-03-27 21:30:00+00:00
17372   2020-03-27 22:00:00+00:00
17373   2020-03-27 22:30:00+00:00
17374   2020-03-27 23:00:00+00:00
17375   2020-03-27 23:30:00+00:00
17376   2020-03-28 00:00:00+00:00
17377   2020-03-28 00:30:00+00:00
17378   2020-03-28 01:00:00+00:00
17379   2020-03-28 01:30:00+00:00
17380   2020-03-28 02:00:00+00:00
17381   2020-03-28 02:30:00+00:00
17382   2020-03-28 03:00:00+00:00
17383   2020-03-28 03:30:00+00:00
17384   2020-03-28 04:00:00+00:00
17385   2020-03-28 04:30:00+00:00
17386   2020-03-28 05:00:00+00:00
17387   2020-03-28 05:30:00+00:00
17388   2020-03-28 06:00:00+00:00
17389   2020-03-28 06:30:00+00:00
17390   2020-03-28 07:00:00+00:00
17391   2020-03-28 07:30:00+00:00
17392   2020-03-28 08:00:00+00:00
17393   2020-03-28 08:30:00+00:00
17394   2020-03-28 09:00:00+00:00
17395   2020-03-28 09:30:00+00:00
17396   2020-03-28 10:00:00+00:00
17397   2020-03-28 10:30:00+00:00
17398   2020-03-28 11:00:00+00:00
17399   2020-0

In [9]:
# Create a new dataframe with the missing timestamps
missing_timestamps = pd.date_range(start='2020-03-29 00:00:00+00:00', end='2020-03-29 01:30:00+00:00', freq='30min')
missing_timestamps = pd.DataFrame(missing_timestamps, columns=['timestamp'])

# Insert the missing timestamps into the df_main
df_main = pd.concat([df_main, missing_timestamps], axis=0)

# Sort the dataframe by timestamp
df_main = df_main.sort_values('timestamp')

# Reset the index of the dataframe
df_main = df_main.reset_index(drop=True)

# Fill the other cells of the inserted rows with NaN
df_main = df_main.fillna(np.nan)
df_main

,timestamp,net_load,month,day,hour,day_of_week,season,avg_net_load,weekend_or_bank_holiday,solar_radiation,sunshine_duration
0,2019-04-01 00:00:00+00:00,198.79,4.0,1.0,1.0,0.0,2.0,152.86,0.0,0.0,0.0
1,2019-04-01 00:30:00+00:00,250.51,4.0,1.0,1.0,0.0,2.0,204.52,0.0,0.0,0.0
2,2019-04-01 01:00:00+00:00,194.07,4.0,1.0,2.0,0.0,2.0,214.66,0.0,0.0,0.0
3,2019-04-01 01:30:00+00:00,245.33,4.0,1.0,2.0,0.0,2.0,209.72,0.0,0.0,0.0
4,2019-04-01 02:00:00+00:00,223.68,4.0,1.0,3.0,0.0,2.0,230.59,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
17561,2020-03-31 20:30:00+00:00,28.10,3.0,31.0,21.0,1.0,2.0,282.00,0.0,NaN,NaN
17562,2020-03-31 21:00:00+00:00,11.10,3.0,31.0,22.0,1.0,2.0,321.01,0.0,NaN,NaN
17563,2020-03-31 21:30:00+00:00,196.48,3.0,31.0,22.0,1.0,2.0,353.42,0.0,NaN,NaN
17564,2020-03-31 22:00:00+00:00,293.14,3.0,31.0,23.0,1.0,2.0,219.42,0.0,NaN,NaN


In [10]:
missing_timestamps

,timestamp
0,2020-03-29 00:00:00+00:00
1,2020-03-29 00:30:00+00:00
2,2020-03-29 01:00:00+00:00
3,2020-03-29 01:30:00+00:00


In [11]:
df_main

,timestamp,net_load,month,day,hour,day_of_week,season,avg_net_load,weekend_or_bank_holiday,solar_radiation,sunshine_duration
0,2019-04-01 00:00:00+00:00,198.79,4.0,1.0,1.0,0.0,2.0,152.86,0.0,0.0,0.0
1,2019-04-01 00:30:00+00:00,250.51,4.0,1.0,1.0,0.0,2.0,204.52,0.0,0.0,0.0
2,2019-04-01 01:00:00+00:00,194.07,4.0,1.0,2.0,0.0,2.0,214.66,0.0,0.0,0.0
3,2019-04-01 01:30:00+00:00,245.33,4.0,1.0,2.0,0.0,2.0,209.72,0.0,0.0,0.0
4,2019-04-01 02:00:00+00:00,223.68,4.0,1.0,3.0,0.0,2.0,230.59,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
17561,2020-03-31 20:30:00+00:00,28.10,3.0,31.0,21.0,1.0,2.0,282.00,0.0,NaN,NaN
17562,2020-03-31 21:00:00+00:00,11.10,3.0,31.0,22.0,1.0,2.0,321.01,0.0,NaN,NaN
17563,2020-03-31 21:30:00+00:00,196.48,3.0,31.0,22.0,1.0,2.0,353.42,0.0,NaN,NaN
17564,2020-03-31 22:00:00+00:00,293.14,3.0,31.0,23.0,1.0,2.0,219.42,0.0,NaN,NaN


In [12]:
# Check the range of the timestamp column
min_timestamp = df_main['timestamp'].min()
max_timestamp = df_main['timestamp'].max()

# Get the last 25 values of the timestamp range
last_25_timestamps = df_main['timestamp'].sort_values().tail(170)
last_25_timestamps

17396   2020-03-28 10:00:00+00:00
17397   2020-03-28 10:30:00+00:00
17398   2020-03-28 11:00:00+00:00
17399   2020-03-28 11:30:00+00:00
17400   2020-03-28 12:00:00+00:00
17401   2020-03-28 12:30:00+00:00
17402   2020-03-28 13:00:00+00:00
17403   2020-03-28 13:30:00+00:00
17404   2020-03-28 14:00:00+00:00
17405   2020-03-28 14:30:00+00:00
17406   2020-03-28 15:00:00+00:00
17407   2020-03-28 15:30:00+00:00
17408   2020-03-28 16:00:00+00:00
17409   2020-03-28 16:30:00+00:00
17410   2020-03-28 17:00:00+00:00
17411   2020-03-28 17:30:00+00:00
17412   2020-03-28 18:00:00+00:00
17413   2020-03-28 18:30:00+00:00
17414   2020-03-28 19:00:00+00:00
17415   2020-03-28 19:30:00+00:00
17416   2020-03-28 20:00:00+00:00
17417   2020-03-28 20:30:00+00:00
17418   2020-03-28 21:00:00+00:00
17419   2020-03-28 21:30:00+00:00
17420   2020-03-28 22:00:00+00:00
17421   2020-03-28 22:30:00+00:00
17422   2020-03-28 23:00:00+00:00
17423   2020-03-28 23:30:00+00:00
17424   2020-03-29 00:00:00+00:00
17425   2020-0

In [13]:
# Check the range of the timestamp column
min_timestamp = df_main['timestamp'].min()
max_timestamp = df_main['timestamp'].max()

# Get the last 25 values of the timestamp range
last_25_timestamps = df_main['timestamp'].sort_values().tail(180)
last_25_timestamps

17386   2020-03-28 05:00:00+00:00
17387   2020-03-28 05:30:00+00:00
17388   2020-03-28 06:00:00+00:00
17389   2020-03-28 06:30:00+00:00
17390   2020-03-28 07:00:00+00:00
17391   2020-03-28 07:30:00+00:00
17392   2020-03-28 08:00:00+00:00
17393   2020-03-28 08:30:00+00:00
17394   2020-03-28 09:00:00+00:00
17395   2020-03-28 09:30:00+00:00
17396   2020-03-28 10:00:00+00:00
17397   2020-03-28 10:30:00+00:00
17398   2020-03-28 11:00:00+00:00
17399   2020-03-28 11:30:00+00:00
17400   2020-03-28 12:00:00+00:00
17401   2020-03-28 12:30:00+00:00
17402   2020-03-28 13:00:00+00:00
17403   2020-03-28 13:30:00+00:00
17404   2020-03-28 14:00:00+00:00
17405   2020-03-28 14:30:00+00:00
17406   2020-03-28 15:00:00+00:00
17407   2020-03-28 15:30:00+00:00
17408   2020-03-28 16:00:00+00:00
17409   2020-03-28 16:30:00+00:00
17410   2020-03-28 17:00:00+00:00
17411   2020-03-28 17:30:00+00:00
17412   2020-03-28 18:00:00+00:00
17413   2020-03-28 18:30:00+00:00
17414   2020-03-28 19:00:00+00:00
17415   2020-0

In [14]:
# Create and insert rows to account for missing values of half hourly data betwen 2020-03-28 23:30:00+00:00 and 2020-03-29 02:00:00+00:00
# Create a new dataframe with the missing timestamps
missing_timestamps = pd.date_range(start='2020-03-29 00:00:00+00:00', end='2020-03-29 01:30:00+00:00', freq='30min')
missing_timestamps = pd.DataFrame(missing_timestamps, columns=['timestamp'])
missing_timestamps

#insert the missing timestamps into the df_main
df_main = pd.concat([df_main, missing_timestamps], axis=0)
df_main

,timestamp,net_load,month,day,hour,day_of_week,season,avg_net_load,weekend_or_bank_holiday,solar_radiation,sunshine_duration
0,2019-04-01 00:00:00+00:00,198.79,4.0,1.0,1.0,0.0,2.0,152.86,0.0,0.0,0.0
1,2019-04-01 00:30:00+00:00,250.51,4.0,1.0,1.0,0.0,2.0,204.52,0.0,0.0,0.0
2,2019-04-01 01:00:00+00:00,194.07,4.0,1.0,2.0,0.0,2.0,214.66,0.0,0.0,0.0
3,2019-04-01 01:30:00+00:00,245.33,4.0,1.0,2.0,0.0,2.0,209.72,0.0,0.0,0.0
4,2019-04-01 02:00:00+00:00,223.68,4.0,1.0,3.0,0.0,2.0,230.59,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
17565,2020-03-31 22:30:00+00:00,346.70,3.0,31.0,23.0,1.0,2.0,193.11,0.0,NaN,NaN
0,2020-03-29 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-29 00:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-29 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Create raws for missing data 
### (between 2020-03-28 23:30:00+00:00 and 2020-03-29 02:00:00+00:00, exclusive)

In [ ]:
#min_date = df_main['timestamp'].min()
#max_date = df_main['timestamp'].max()

#date_range = pd.date_range(start=min_date, end=max_date, freq='30min')

# new df main inserting all the timestamps in date range
#df_main = df_main.set_index('timestamp').reindex(date_range).rename_axis('timestamp').reset_index()